# Introduction

At the onset of the 2016 fantasy football season I decided that I was going to try and win my league with numbers and computer science. I was going to take the opportunity to learn modern data science tools and finally put some of that college statistics I'd learned to use. 

A combination of factors led me to move through various tools and approaches at the beginning of the season. Ultimately I ended up using Excel to quickly fire up a list of players in the order I wanted by hand. In the next sectoin I'm goign to reproduce this methdology using python, specifically the data science package `pandas`.

# Python Reproduction of Excel Methodology

## Imports

These are the packages used to reproduce the list I used for drafting my team for 2016.

* `pandas` Is a Python data analysis library and is available [here](http://pandas.pydata.org/)
* `numpy` Is a package for scientific computing in python, used below primarily for its mathmatical functions and constructs. It is available [here](http://www.numpy.org/).
* `matplotlib.pyplot` Is a package for plotting data and is available [here](http://matplotlib.org/).
* `pyffl` Is a package I've developed for scraping fantasy football data and will be including any pure python functions there. It is available [here](https://github.com/smkell/pyffl)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display

import pyffl

## Getting the data

My original intent was to use a wide variety of projection and ranking data to formulate my ranks. However, time, laziness and indecisiveness ulitmately led me to only using ESPN's standard projections. 

I've created a function in the `pyffl` package for retrieving projections from a variety of sources. Of course once again only `ESPN` is currently implemented.

In [2]:
rules = pyffl.LeagueRules()

In [3]:
projections = pyffl.scrape_projections(['espn'], 2016)


Scraping ESPN projections for week 0 of 2016 season
...Done


In [4]:
projections = pyffl.calculate_points(projections, rules)

## Building the dataframe

Pandas primarily operates on objects known as `Series` and `DataFrame` where a `DataFrame` is a table composed of several `Series` associated together by an `index`. In the below code segment we construct a `DataFram` for our projections. The `*_key` lists give the names of the columns in the desired order for display.

In [5]:
df = pd.DataFrame(projections)
df['source'] = 'espn'
df['projection'] = True
df['vor'] = np.nan
df['rank'] = df['pts'].rank(ascending=0)
df['positionRank'] = 0.0

info_keys = ['rank', 'positionRank', 'name', 'team', 'position', 'source', 'projection']
skill_keys = ['passCmp', 'passAtt', 'passYds', 'passTds', 'passInts',
              'rushAtt', 'rushYds', 'rushTds',
              'recsCmp', 'recsAtt', 'recsYds', 'recsTds']
dst_keys = ['dstTckls', 'dstSacks', 'dstFmblFrc', 'dstFmblRec', 'dstInts', 'dstIntTds', 'dstFmblTds']
k_keys = ['fg0139Cmp', 'fg0139Att', 'fg4049Cmp', 'fg4049Att', 'fg50Cmp', 'fg50Att',
          'fgCmp', 'fgAtt', 'xpCmp', 'xpAtt']
calc_keys = ['pts','vor']
all_keys = info_keys + skill_keys + dst_keys + calc_keys
df = df[all_keys]

## Calculating VOR 

`Value over replacement` is a measure of a player's value compared to other players in that position. The theory here is that we can measure how valuable a player by comparing how many more points he is projected to score than the next startable player in the position. Analysing this number shows descrete gaps in value where players are split in tiers. We can, in principle, use this information to pick the most valuable players at the right moment in the draft.

In [6]:
# Positions is a `dict` where the key is the position, and the value is the number of players in that position which could
# be started in any given week. I.e. if there are 8 teams in the league and one QB slot per team then 8 QBs could be started.
# Likewise if there are 2 RB slots and 1 RB/WR/TE Flex then at most 3*8(24) RBs could be started in a given week.
positions = {
    'QB': (rules.starting_qbs + rules.starting_superflex) * rules.num_teams,
    'RB': (rules.starting_rbs + rules.starting_superflex + rules.starting_flex) * rules.num_teams,
    'WR': (rules.starting_wrs + rules.starting_superflex + rules.starting_flex) * rules.num_teams,
    'TE': (rules.starting_tes + rules.starting_superflex + rules.starting_flex) * rules.num_teams,
    'K': rules.starting_ks * rules.num_teams,
    'D/ST': rules.starting_dst * rules.num_teams
}

In [7]:
# Iterate through the positions, and calculate the vor for each player in the position.
for position, draftable in positions.iteritems():
    last_qb = df[df['position'] == position].sort_values(by='pts',ascending=False).iloc[draftable,:]
    df.ix[df['position'] == position, 'vor'] = df.ix[df['position'] == position, 'pts'] - last_qb['pts']
    df.ix[df['position'] == position, 'positionRank'] = df.ix[df['position'] == position, 'pts'].rank(ascending=0)

The following table are the top 16 players in all positions. It represents my rankings for the first two rounds of the 2016 draft.

In [8]:
round = 0
start = round * rules.num_teams
end = start + rules.num_teams
df[['rank', 'positionRank', 'name', 'position', 'pts', 'vor']].sort_values(by='vor',ascending=False).iloc[start:end,:]

,rank,positionRank,name,position,pts,vor
320,1.0,1.0,Antonio Brown,WR,383.00,185.42
322,2.0,2.0,Julio Jones,WR,346.15,148.57
520,47.0,1.0,Rob Gronkowski,TE,244.19,140.42
521,55.0,2.0,Jordan Reed,TE,234.27,130.50
123,13.0,1.0,David Johnson,RB,280.66,120.97
124,19.0,2.0,Devonta Freeman,RB,270.52,110.83
121,22.0,3.0,Todd Gurley,RB,269.01,109.32
321,6.0,3.0,Odell Beckham Jr.,WR,305.81,108.23


In [9]:
round = 1
start = 1 + (round * rules.num_teams)
end = start + rules.num_teams
df[['rank', 'name', 'position', 'pts', 'vor']].sort_values(by='vor',ascending=False).iloc[start:end,:]

,rank,name,position,pts,vor
120,26.0,Adrian Peterson,RB,264.50,104.81
524,79.0,Travis Kelce,TE,198.89,95.12
130,36.0,LeSean McCoy,RB,253.65,93.96
125,37.0,Le'Veon Bell,RB,252.79,93.10
523,85.0,Delanie Walker,TE,196.52,92.75
126,39.0,Lamar Miller,RB,251.80,92.11
122,40.0,Ezekiel Elliott,RB,251.62,91.93
323,10.0,DeAndre Hopkins,WR,284.82,87.24


# A quick interlude for analysis

So far we've actually come pretty far in terms of data collection and massaging. I think, then that it's time that we start taking a look at what it all means. 

In the above table I've listed the 16 most valuable players according to my model. There are problems here. First of all going into the draft I never had any intention of drafting either Rob Gronkowski or Jordan Reed, simply based off the fact that I *knew* that the tight-end position simply wasn't valuable enough to justify a first or second round pick. The fact that two tightends show up in the top 16 suggest that either my assumptions, or the model, are wrong.

## Comparing projected rankings to actual rankings

In [10]:
actuals = pyffl.scrape_actuals(2016)
actuals = pyffl.calculate_points(actuals, rules)

In [11]:
dfa = pd.DataFrame(actuals, columns=all_keys)
dfa['vor'] = np.nan
dfa['rank'] = dfa['pts'].rank(ascending=0)

In [12]:
positions = {
    'QB': (rules.starting_qbs + rules.starting_superflex) * rules.num_teams,
    'RB': (rules.starting_rbs + rules.starting_superflex + rules.starting_flex) * rules.num_teams,
    'WR': (rules.starting_wrs + rules.starting_superflex + rules.starting_flex) * rules.num_teams,
    'TE': (rules.starting_tes + rules.starting_superflex + rules.starting_flex) * rules.num_teams,
}
# Iterate through the positions, and calculate the vor for each player in the position.
for position, draftable in positions.iteritems():
    last_qb = dfa[dfa['position'] == position].sort_values(by='pts',ascending=False).iloc[draftable,:]
    dfa.ix[dfa['position'] == position, 'vor'] = dfa.ix[dfa['position'] == position, 'pts'] - last_qb['pts']
    dfa.ix[dfa['position'] == position, 'positionRank'] = dfa.ix[dfa['position'] == position, 'pts'].rank(ascending=0)

### Round 1 Projected vs Actual

In [13]:
round = 0
start = round * rules.num_teams
end = start + rules.num_teams
display(df[['rank', 'positionRank', 'name', 'position', 'pts', 'vor']].sort_values(by='vor', ascending=False).iloc[start:end,:])
display(dfa[['rank', 'positionRank', 'name', 'position', 'pts', 'vor']].sort_values(by='vor',ascending=False).iloc[start:end,:])

,rank,positionRank,name,position,pts,vor
320,1.0,1.0,Antonio Brown,WR,383.00,185.42
322,2.0,2.0,Julio Jones,WR,346.15,148.57
520,47.0,1.0,Rob Gronkowski,TE,244.19,140.42
521,55.0,2.0,Jordan Reed,TE,234.27,130.50
123,13.0,1.0,David Johnson,RB,280.66,120.97
124,19.0,2.0,Devonta Freeman,RB,270.52,110.83
121,22.0,3.0,Todd Gurley,RB,269.01,109.32
321,6.0,3.0,Odell Beckham Jr.,WR,305.81,108.23


,rank,positionRank,name,position,pts,vor
267,1.0,1.0,David Johnson,RB,286.4,185.6
223,2.0,2.0,Ezekiel Elliott,RB,266.7,165.9
183,5.0,3.0,DeMarco Murray,RB,250.6,149.8
114,11.0,4.0,Melvin Gordon,RB,230.3,129.5
302,7.0,1.0,Antonio Brown,WR,242.7,106.3
33,10.0,2.0,Mike Evans,WR,235.0,98.6
422,22.0,5.0,Le'Veon Bell,RB,194.6,93.8
53,28.0,6.0,LeSean McCoy,RB,188.4,87.6


### Round 2 Projected vs. Actual

In [14]:
round = 1
start = 1 + (round * rules.num_teams)
end = start + rules.num_teams
display(df[['rank', 'positionRank', 'name', 'position', 'pts', 'vor']].sort_values(by='vor', ascending=False).iloc[start:end,:])
display(dfa[['rank', 'positionRank', 'name', 'position', 'pts', 'vor']].sort_values(by='vor',ascending=False).iloc[start:end,:])

,rank,positionRank,name,position,pts,vor
120,26.0,4.0,Adrian Peterson,RB,264.50,104.81
524,79.0,4.0,Travis Kelce,TE,198.89,95.12
130,36.0,5.0,LeSean McCoy,RB,253.65,93.96
125,37.0,6.0,Le'Veon Bell,RB,252.79,93.10
523,85.0,5.0,Delanie Walker,TE,196.52,92.75
126,39.0,7.0,Lamar Miller,RB,251.80,92.11
122,40.0,8.0,Ezekiel Elliott,RB,251.62,91.93
323,10.0,4.0,DeAndre Hopkins,WR,284.82,87.24


,rank,positionRank,name,position,pts,vor
92,3.0,1.0,Aaron Rodgers,QB,263.46,81.56
293,61.0,2.0,Jordan Reed,TE,152.00,81.40
184,66.0,3.0,Delanie Walker,TE,151.00,80.40
123,4.0,2.0,Drew Brees,QB,261.34,79.44
210,73.0,4.0,Jimmy Graham,TE,145.60,75.00
19,37.0,7.0,Devonta Freeman,RB,174.80,74.00
24,13.0,3.0,Julio Jones,WR,209.00,72.60
109,77.0,5.0,Travis Kelce,TE,142.00,71.40


### Average projected points per position vs Average actual points per position

In [15]:
num_rostered = (rules.starting_qbs + 
                rules.starting_rbs + 
                rules.starting_wrs + 
                rules.starting_tes + 
                rules.starting_ks + 
                rules.starting_dst + 
                rules.starting_flex +
                rules.starting_superflex +
                8)
grp_proj = df.sort_values(by='pts', ascending=False).iloc[0:num_rostered*8].groupby('position')
display(grp_proj.agg({'pts': np.mean}))

grp_actl = dfa.sort_values(by='pts', ascending=False).iloc[0:num_rostered*8].groupby('position')
display(grp_actl.agg({'pts': np.mean}))

,pts
position,
QB,262.582333
RB,201.517368
TE,180.324375
WR,207.957333


,pts
position,
QB,184.534667
RB,149.808571
TE,118.972222
WR,143.944918


### Identifying the biggest busts of the year.

The below table merges the actual and projected points, and calculates the difference between the two. The table is then sorted by the difference to identify the biggest "busts" or disappointments. Several of these we can attribute to injuries and we can discount them fairly easily. Others are less easy to explain.

In [17]:
def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

mrg = df.merge(dfa, left_on=['name','position'], right_on=['name','position'], how='inner', suffixes=['_proj', '_actl'])
mrg['pts_diff'] = mrg['pts_actl'] - mrg['pts_proj']
mrg['positionRank_diff'] = mrg['positionRank_proj'] - mrg['positionRank_actl']
mrg_cols = ['name', 'position','positionRank_proj', 'positionRank_actl', 'pts_proj', 'pts_actl', 'pts_diff', 'positionRank_diff']
tbl = mrg[mrg_cols].sort_values(by='pts_diff').iloc[0:rules.num_teams*3,:]
display(tbl.style.applymap(color_negative_red))

## Point Dropoff By Position

This chart displays the number of points missed out on by drafting the last starter in the position, rather than the best. "The Fantasy Football Guys" call this the magic formula.

In [31]:
grp_proj = dfa.sort_values(by='vor', ascending=False).groupby('position')
tbl = grp_proj.agg({'vor': np.max}).sort_values(by='vor', ascending=False)
tbl['vor_per_week'] = tbl['vor'] / 13
display(tbl)

,vor,vor_per_week
position,,
RB,185.60,14.276923
WR,106.30,8.176923
TE,84.40,6.492308
QB,81.56,6.273846
K,NaN,NaN


This fairly straightforward table provides a fair amount of valuable information.

In [91]:

winning_scores = pd.DataFrame([
        { 'week': 1, 'home': 'KELL', 'away': 'PINK', 'home_score': 115.3, 'away_score': 111.7},
        { 'week': 1, 'home': 'ARTH', 'away': 'TAUR', 'home_score': 117.8, 'away_score': 117.7},
        { 'week': 1, 'home': 'OBRY', 'away': 'TUCK', 'home_score': 202.1, 'away_score': 138.6},
        { 'week': 1, 'home': 'HMBE', 'away': 'SING', 'home_score': 150.6, 'away_score': 110.5},
        { 'week': 2, 'home': 'ARTH', 'away': 'TUCK', 'home_score': 115.9, 'away_score': 97.3},
        { 'week': 2, 'home': 'PINK', 'away': 'TAUR', 'home_score': 100.3, 'away_score': 151.4},
        { 'week': 2, 'home': 'HMBE', 'away': 'OBRY', 'home_score': 117, 'away_score': 100.3},
        { 'week': 2, 'home': 'TUCK', 'away': 'SING', 'home_score': 136.4, 'away_score': 90.4},
        { 'week': 3, 'home': 'KELL', 'away': 'TAUR', 'home_score': 114.3, 'away_score': 167},
        { 'week': 3, 'home': 'PINK', 'away': 'ARTH', 'home_score': 126.7, 'away_score': 119.6},
        { 'week': 3, 'home': 'OBRY', 'away': 'SING', 'home_score': 92.7, 'away_score': 159.4},
        { 'week': 3, 'home': 'TUCK', 'away': 'HMBE', 'home_score': 100.5, 'away_score': 140.8},
        { 'week': 4, 'home': 'OBRY', 'away': 'KELL', 'home_score': 110.5, 'away_score': 84.3},
        { 'week': 4, 'home': 'TUCK', 'away': 'PINK', 'home_score': 183.4, 'away_score': 124.3},
        { 'week': 4, 'home': 'HMBE', 'away': 'ARTH', 'home_score': 151, 'away_score': 96.1},
        { 'week': 4, 'home': 'SING', 'away': 'TAUR', 'home_score': 109.4, 'away_score': 76.7},
        { 'week': 4, 'home': 'SING', 'away': 'TAUR', 'home_score': 109.4, 'away_score': 76.7},
        { 'week': 5, 'home': 'KELL', 'away': 'SING', 'home_score': 122.1, 'away_score': 132.6},
        { 'week': 5, 'home': 'PINK', 'away': 'OBRY', 'home_score': 144.6, 'away_score': 88.7},
        { 'week': 5, 'home': 'ARTH', 'away': 'TUCK', 'home_score': 154.8, 'away_score': 120.6},
        { 'week': 5, 'home': 'TAUR', 'away': 'HMBE', 'home_score': 156.7, 'away_score': 163.3},
        { 'week': 6, 'home': 'HMBE', 'away': 'KELL', 'home_score': 89.7, 'away_score': 139.2},
        { 'week': 6, 'home': 'OBRY', 'away': 'ARTH', 'home_score': 132.8, 'away_score': 138.4},
        { 'week': 6, 'home': 'TUCK', 'away': 'TAUR', 'home_score': 156.4, 'away_score': 130.5},
        { 'week': 6, 'home': 'SING', 'away': 'PINK', 'home_score': 119.1, 'away_score': 110.4},
        { 'week': 7, 'home': 'KELL', 'away': 'TUCK', 'home_score': 114.5, 'away_score': 133.2},
        { 'week': 7, 'home': 'TAUR', 'away': 'OBRY', 'home_score': 97.5, 'away_score': 124.4},
        { 'week': 7, 'home': 'PINK', 'away': 'HMBE', 'home_score': 130.7, 'away_score': 133.3},
        { 'week': 7, 'home': 'ARTH', 'away': 'SING', 'home_score': 148.5, 'away_score': 95.4}
    ], columns=['week', 'home', 'away', 'home_score', 'away_score'])
winning_scores['winning_score'] = winning_scores.loc[:, ['home_score', 'away_score']].max(axis=1)
winning_scores['margin_of_victory'] = winning_scores['winning_score'] - winning_scores.loc[:, ['home_score', 'away_score']].min(axis=1)
display(winning_scores)

,week,home,away,home_score,away_score,winning_score,margin_of_victory
0,1,KELL,PINK,115.3,111.7,115.3,3.6
1,1,ARTH,TAUR,117.8,117.7,117.8,0.1
2,1,OBRY,TUCK,202.1,138.6,202.1,63.5
3,1,HMBE,SING,150.6,110.5,150.6,40.1
4,2,ARTH,TUCK,115.9,97.3,115.9,18.6
5,2,PINK,TAUR,100.3,151.4,151.4,51.1
6,2,HMBE,OBRY,117.0,100.3,117.0,16.7
7,2,TUCK,SING,136.4,90.4,136.4,46.0
8,3,KELL,TAUR,114.3,167.0,167.0,52.7
9,3,PINK,ARTH,126.7,119.6,126.7,7.1


In [95]:
grp_proj = winning_scores.groupby('week')
col_names = {'winning_score': 'Winning Score', 
             'margin_of_victory': 'Margin of Victory'}
display(grp_proj.agg({'winning_score': [np.mean, np.min, np.max],
                      'margin_of_victory': [np.mean, np.min, np.max]
                     }).rename(columns=col_names))

Winning Score               Margin of Victory            
              mean   amin   amax              mean  amin  amax
week                                                          
1          146.450  115.3  202.1            26.825   0.1  63.5
2          130.175  115.9  151.4            33.100  16.7  51.1
3          148.475  126.7  167.0            41.700   7.1  66.7
4          132.740  109.4  183.4            41.120  26.2  59.1
5          148.825  132.6  163.3            26.800   6.6  55.9
6          138.275  119.1  156.4            22.425   5.6  49.5
7          134.850  124.4  148.5            25.325   2.6  53.1

In [101]:
mean_margin_of_victory = grp_proj['margin_of_victory'].mean().mean()
stdev_margin_of_victory = grp_proj['margin_of_victory'].mean().std()
print(mean_margin_of_victory, stdev_margin_of_victory)
tbl['pct_margin_victory'] = (tbl['vor_per_week'] / mean_margin_of_victory) * 100
display(tbl)

(31.04214285714286, 7.7686211009112718)


,vor,vor_per_week,pct_margin_victory
position,,,
RB,185.60,14.276923,45.992067
WR,106.30,8.176923,26.341362
TE,84.40,6.492308,20.914496
QB,81.56,6.273846,20.210738
K,NaN,NaN,NaN
